In [1]:
import tensorflow as tf
import os
import tensorflow_datasets as tfds

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

ValueError: Please provide a TPU Name to connect to.

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
import random
import glob
import cv2
import numpy as np
from pathlib import Path

In [ ]:
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

In [ ]:
directory_path='/gdrive/My Drive/Yedek/Video_Dataset'

In [ ]:
subset_paths={'train':directory_path+'/train','test':directory_path+'/test'}

In [ ]:
def frames_from_video_file(video_path):
  result = []
  src = cv2.VideoCapture(str(video_path))

  for _ in range(20):
    ret, frame = src.read()
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    result.append(frame)

  src.release()
  result = np.array(result)[..., [2, 1, 0]]
  return result

In [ ]:
class FrameGenerator:
  def __init__(self, path, training = False):

    self.path = path
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
train_path = Path(subset_paths['train'])
test_path = Path(subset_paths['test'])

In [ ]:
fg = FrameGenerator(train_path, training=True)
print(fg.class_ids_for_name)

In [ ]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.float32))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_path, training=True),
                                          output_signature = output_signature)

In [ ]:
val_ds = tf.data.Dataset.from_generator(FrameGenerator(test_path),
                                          output_signature = output_signature)

In [ ]:
train_ds = train_ds.batch(4)
val_ds = val_ds.batch(4)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

In [ ]:
with strategy.scope():
  net = tf.keras.applications.EfficientNetB0(include_top = False)
  net.trainable = False

  model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=255),
    tf.keras.layers.TimeDistributed(net),
    tf.keras.layers.Dense(10),
    tf.keras.layers.GlobalAveragePooling3D()
    ])

  model.compile(optimizer = 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

model.fit(train_ds,
          epochs = 10)